In [17]:
# import of the needed packages
import pandas as pd
import numpy as np

In [18]:
# loading the needed data file
data = pd.read_csv('Data\\cumulated_data_bl_final_version.csv', delimiter=',')
# adding a column for the bookmaker predictions
data['Betting_Prediction'] = ''

In [19]:
# list of the bookmakers with complete data for all seasons
betting_providers = ['B365H','B365D','B365A','BWH','BWD','BWA','IWH','IWD','IWA','VCH','VCD','VCA']

In [20]:
# function to transfer odds into probability
def odds_to_probaility_1(odds):
    if odds >= 0:
        probability = odds/(odds+100) # formula for 
    else:
        probability = odds/(odds-100)
    return probability

In [21]:
# function to transfer odds into probability
def odds_to_probaility_2(odds):
    probability = (1/odds)*100 # formula for the transfer
    return probability

In [22]:
# function to transfer the odds into probability for each matchday
def calculate_probs_gameday(data):
    x, z = 0, 0 # variables for the loops
    prob_spieltag, prob_ges = [], [] # lists to safe the probabilities
    while x < len(data): # loop to iterate trough the matchdays
        while z < len(betting_providers): # loop to iterate trough the bookmakers
            prob = odds_to_probaility_2(data[betting_providers[z]][x]) # calling the odds to probability function
            prob_spieltag.append(prob) # adding the calculated probabilities for the matchday
            z += 1
        prob_ges.append(prob_spieltag) # adding the probabilities from the matchday to the list with all matchdays
        prob_spieltag = [] # resetting the list for the matchday
        z = 0 # resetting the counter for the bookmakers
        x += 1
    return prob_ges

In [23]:
# function to calculate the average for a matchday
def calculate_average_prob_gameday(prob_ges):
    z, home, draw, away = 0, 0, 0, 0 # variables for the counter and the average probabilities
    prob_spieltag_schnitt, prob_ges_schnitt = [], [] # lists to save the average probabilities
    for x in prob_ges: # loop to iterate trough the matchdays
        while z < len(x): # loop to iterate trhough the probabilities
            if z % 3 == 0: # if and % 3 = 0 to get the probabilities for a home win, because the first probability from each bookmaker is for a home win
                home = home + x[z] # summing up the bookmakers probabilities for home win
            if z % 3 == 1: # if and % 3 = 1 to get the probabilities for a draw, because the second probability from each bookmaker is for a draw
                draw = draw + x[
                    z]  # summing up the bookmakers probabilities for draw
            if z % 3 == 2: # if and % 3 = 2 to get the probabilities for a away win, because the third probability from each bookmaker is for a away win
                away = away + x[
                    z]  # summing up the bookmakers probabilities for away win
            z += 1
        z = 0
        prob_spieltag_schnitt.append(home / 4) # calculating the average for home win and adding it to the list for the matchday
        prob_spieltag_schnitt.append(
            draw / 4
        )  # calculating the average for draw and adding it to the list for the matchday
        prob_spieltag_schnitt.append(
            away / 4
        )  # calculating the average for away win and adding it to the list for the matchday
        prob_ges_schnitt.append(prob_spieltag_schnitt) # adding the average probabilities for a matchday to the list with all matchdays
        prob_spieltag_schnitt = [] # resetting the list for the matchday
        home, draw, away = 0, 0, 0 # resetting the values for the matchday
    return prob_ges_schnitt


In [24]:
# fucntion to get the prediction of the bookmakers
def get_odd_prediction(prob_ges_schnitt):
    erg_odds, erg_prob = [], [] # lists for storing the maximum probability and the predcition
    for x in prob_ges_schnitt: # loop to iterate trough the matchdays
        max_value = max(x) # getting the maximum probability for an match day
        erg_prob.append(max_value) # adding the maximum probability to the list for all matchdays
        # getting the predictions by checking the index of the maximum
        if x.index(max_value) == 0: # if the maximum is on the first position, the prediction is home win
            erg_odds.append('H') # adding the prediction to the list for all matchdays
        if x.index(
                max_value
        ) == 1:  # if the maximum is on the second position, the prediction is draw
            erg_odds.append(
                'D')  # adding the prediction to the list for all matchdays
        if x.index(
                max_value
        ) == 2:  # if the maximum is on the thrid position, the prediction is away win
            erg_odds.append(
                'A')  # adding the prediction to the list for all matchdays
    return erg_odds, erg_prob

In [25]:
# function to add the predcitions and their probabilities to the dataframe
def add_odd_predcition_to_df(data, erg_odds, erg_prob):
    data['Betting_Prediction'] = erg_odds # adding the predictions to the dataframe
    data['Betting_Prediction_Probability'] = erg_prob # adding the probabilities from the predictions to the dataframe
    return data

In [26]:
def add_maximum_odds_to_df(data):
    max_value_home, max_value_away, max_value_draw = [], [], [] # lists for saving the highest odds for an event
    for index, rows in data.iterrows(): # loop to iterate trough the matchdays
        max_value_home = max(rows['B365H'], rows['BWH'], rows["IWH"], rows['VCH']) # adding the maximum value for home win to the list
        max_value_away = max(rows['B365A'], rows['BWA'], rows["IWA"], rows['VCA']) # adding the maximum value for away win to the list
        max_value_draw = max(
            rows['B365D'], rows['BWD'], rows["IWD"],
            rows['VCD'])  # adding the maximum value for draw to the list
    data['MaxHome'] = max_value_home # adding the maximum odds for home win to the dataframe
    data['MaxAway'] = max_value_away # adding the maximum odds for away win to the dataframe
    data[
        'MaxDraw'] = max_value_draw  # adding the maximum odds for draw to the dataframe
    return data

In [27]:
# function to reduce the dataframe to only the necessary columns
def reduce_dataframe(data):
    data_reduced = data[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
        'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA',
        'VCH', 'VCD', 'VCA', 'Betting_Prediction', 'Betting_Prediction_Probability', 'MaxHome',
        'MaxAway', 'MaxDraw'
    ]] # only selecting the necessary columns
    return data_reduced


In [28]:
# function to save the dataframe as a csv file
def df_to_csv(data):
    data.to_csv("Data\\dataframe_with_odd_predictions_v4.csv")

In [29]:
# function to get the bookmakers performance
def evaluate_odd_performance(data):
    z, right_prediction, false_prediction = 0, 0, 0 # variables for the counter, right and false predictions
    while z < len(data): # loop for iterating trough the dataframe
        if data.FTR[z] == data.Betting_Prediction[z]: # checking if the result from the match is equal to the bookmakers prediction
            right_prediction += 1 # adding 1 if the predcition matches the result from the game
        else:
            false_prediction += 1 # adding 1 if the predcition doesn´t match the result from the game
        z += 1
    return right_prediction, false_prediction

In [30]:
# function to show the bookmakers performance
def show_odd_performance(data, right_prediction, false_prediction):
    print('Right Predictions: ', right_prediction) # number of the right predictions
    print('False Predictions: ', false_prediction) # number of the false predictions
    print('Number of Games: ', len(data)) # numbe rof all matches
    print('Accuracy: ', round(right_prediction/len(data)*100,2), '%') # calculation of the accuracy of the bookmakers


In [31]:
# function to run all functions with one click
def run_all(data):
    prob_ges = calculate_probs_gameday(data)
    prob_ges_schnitt = calculate_average_prob_gameday(prob_ges)
    erg_odds, erg_prob = get_odd_prediction(prob_ges_schnitt)
    data = add_odd_predcition_to_df(data, erg_odds, erg_prob)
    data = add_maximum_odds_to_df(data)
    data = reduce_dataframe(data)
    df_to_csv(data)
    right_prediction, false_prediction = evaluate_odd_performance(data)
    show_odd_performance(data, right_prediction, false_prediction)
run_all(data)

Right Predictions:  2366
False Predictions:  2224
Number of Games:  4590
Accuracy:  51.55 % #
